In [1]:
import nbimporter; nbimporter.options["only_defs"] = False
from P0 import compileString

def runpywasm(wasmfile):
    import pywasm
    def write(s, i): print(i)
    def writeln(s): print('\n')
    def read(s): return int(input())
    vm = pywasm.load(wasmfile, {'P0lib': {'write': write, 'writeln': writeln, 'read': read}})

In [3]:
def runwasm(wasmfile):
    from IPython.core.display import display, Javascript
    display(Javascript("""
    const params = { 
        P0lib: { 
            write: i => this.append_stream({text: '' + i, name: 'stdout'}),
            writeln: () => this.append_stream({text: '\\n', name: 'stdout'}),
            read: () => window.prompt()
        }
    }

    fetch('""" + wasmfile + """') // asynchronously fetch file, return Response object
      .then(response => response.arrayBuffer()) // read the response to completion and stores it in an ArrayBuffer
      .then(code => WebAssembly.compile(code)) // compile (sharable) code.wasm
      .then(module => WebAssembly.instantiate(module, params)) // create an instance with memory
    // .then(instance => instance.exports.program()); // run the main program; not needed if start function specified
    """))

In [6]:
from wasmer import engine, Store, Module, Instance, ImportObject, Function
from wasmer_compiler_cranelift import Compiler

def runwasmer(wasmfile):
    def write(i: int): print(i)
    def writeln(): print('\n')
    def read() -> int: return int(input()) 
    store = Store(engine.JIT(Compiler))
    module = Module(store, open(wasmfile, 'rb').read())
    import_object = ImportObject()
    import_object.register("P0lib", {"write": Function(store, write),
                                     "writeln": Function(store, writeln),"read": Function(store, read)})
    instance = Instance(module, import_object)

ImportError: Wasmer is not available on this system

In [2]:
compileString("""
type Arr = [0 .. 5] → integer
type Set = set[0..5]
program repeatprimes
    var sorted:boolean
    var i:integer
    var a:Arr
        a[0] := 1
        a[1] := 2
        a[2] := 3
        a[3] := 4
        a[4] := 5
        a[5] := 6
        a := [2 × 2 for i ∈ 0 .. 5]
        write(a[0])
        write(a[1])
        write(a[2])
        write(a[3])
        write(a[4])
        write(a[5])  
        

""", 'test.wat', target = 'wat')

In [3]:
!wat2wasm test.wat
runwasm("test.wasm")

test.wat:103:1: error: opcode not allowed: memory.copy
memory.copy
^^^^^^^^^^^


NameError: name 'runwasm' is not defined

Example of what needs to be compield:
        a[0] := 1
        a[1] := 2
        a[2] := 3
        a[3] := 4
        a[4] := 5
        a[5] := 6
        sorted := some i ∈ 0 .. 4 • a[i] = 6

        a := [i ∈ 0 .. 5 • i × i]

a := {i for i ∈ 0 .. 5 if i mod 2 = 1} 
b := {i ∈ 0 .. 5 | i mod 2 = 1 • i} 


        a[0] := 7
        a[1] := 8
        a[2] := 9
        a[3] := 10
        a[4] := 11
        a[5] := 12
        a := [i ∈ 0 .. 5 • i × i]
        write(a[0])
        write(a[1])
        write(a[2])
        write(a[3])
        write(a[4])
        write(a[5])  

In [4]:
!cat test.wat #This is for printing text output of wat file

(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(global $_memsize (mut i32) i32.const 0)
(func $program
(local $sorted i32)
(local $i i32)
(local $a i32)
(local $b i32)
(local $0 i32)
(local $_fp i32)
global.get $_memsize
local.set $_fp
global.get $_memsize
local.tee $a
i32.const 24
i32.add
global.set $_memsize
global.get $_memsize
local.tee $b
i32.const 24
i32.add
global.set $_memsize
local.get $a
local.get $b
i32.const 24
memory.copy
local.get $_fp
global.set $_memsize
)
(memory 1)
(start $program)
)